## Correlation matrices and eigenvalues of per-core logs:

In [1]:
import os
import pandas as pd
from numpy.linalg import eig, eigvalsh

import eigen_analysis

In [2]:
def get_latencies(out_fname):
    with open(out_fname, 'r') as f:
        lines = f.readlines()
    header = lines[0].rstrip('\n').split()
    read_lat = lines[1].rstrip('\n').split()
    lat = {'read': dict(zip(header[1:], [float(y) for y in read_lat[1:]]))}

    return lat['read']

In [3]:
def get_joules(df):
    joules_sum = df['joules_diff'].sum()
    joules = {'joules_sum': joules_sum}
    
    return joules

### Correlation Matrices of (instructions_diff, cycles_diff, ref_cycles_diff, llc_miss_diff):

In [31]:
corr_dvfs_counter_list = {}
corr_dvfs_non_counter_list = {}
joules_dvfs_list = {}

for dvfs in ['0xd00', '0x1d00']:
    corr_qps_counter_list = {}
    corr_qps_non_counter_list = {}
    joules_qps_list = {}

    for qps in ['200k', '400k', '600k']:
        corr_itr_counter_list = {}
        corr_itr_non_counter_list = {}
        joules_itr_list = {}

        all_itr_counter_dir = qps + '_' + dvfs + '_counters_merged/'
        all_itr_non_counter_dir = qps + '_' + dvfs + '_non_counters_merged/'

        print('Parsing counter metrics in ' + all_itr_counter_dir)
        for file in os.listdir(all_itr_counter_dir):
            if os.path.isdir(all_itr_counter_dir + file):
                continue
            itr = file.split('_')[2]
            df = pd.read_csv(all_itr_counter_dir + file, sep = ',')
            joules = get_joules(df.copy())
            joules_itr_list[itr] = joules
            print('ITR_DELAY: ', itr, joules)
            df = df.drop([df.columns[0], 'joules_diff', 'timestamp'], axis=1)
            df_corr = df.corr()
            corr_itr_counter_list[itr] = df_corr

        print('Parsing non-counter metrics in ' + all_itr_non_counter_dir)
        for file in os.listdir(all_itr_non_counter_dir):
            if os.path.isdir(all_itr_non_counter_dir + file):
                continue
            itr = file.split('_')[2]
            print('ITR_DELAY: ', itr)
            df = pd.read_csv(all_itr_non_counter_dir + file, sep= ',')
            df_corr = df.drop([df.columns[0], 'timestamp'], axis = 1).corr()
            corr_itr_non_counter_list[itr] = df_corr
            
        corr_qps_counter_list[qps] = corr_itr_counter_list
        corr_qps_non_counter_list[qps] = corr_itr_non_counter_list
        joules_qps_list[qps] = joules_itr_list
        
    corr_dvfs_counter_list[dvfs] = corr_qps_counter_list
    corr_dvfs_non_counter_list[dvfs] = corr_qps_non_counter_list
    joules_dvfs_list[dvfs] = joules_qps_list

Parsing counter metrics in 200k_0xd00_counters_merged/
ITR_DELAY:  200 {'joules_sum': 34341.416562000035}
ITR_DELAY:  400 {'joules_sum': 33724.28090399999}
ITR_DELAY:  350 {'joules_sum': 34207.359545999956}
ITR_DELAY:  100 {'joules_sum': 34939.679369000085}
ITR_DELAY:  250 {'joules_sum': 34107.859213000156}
ITR_DELAY:  300 {'joules_sum': 34050.92852300004}
Parsing non-counter metrics in 200k_0xd00_non_counters_merged/
ITR_DELAY:  400
ITR_DELAY:  300
ITR_DELAY:  350
ITR_DELAY:  250
ITR_DELAY:  200
ITR_DELAY:  100
Parsing counter metrics in 400k_0xd00_counters_merged/
ITR_DELAY:  300 {'joules_sum': 37753.34609300003}
ITR_DELAY:  100 {'joules_sum': 39215.126425000024}
ITR_DELAY:  400 {'joules_sum': 37394.88526599988}
ITR_DELAY:  250 {'joules_sum': 38249.420906999876}
ITR_DELAY:  200 {'joules_sum': 38266.06536700008}
ITR_DELAY:  350 {'joules_sum': 37007.07001900003}
Parsing non-counter metrics in 400k_0xd00_non_counters_merged/
ITR_DELAY:  250
ITR_DELAY:  400
ITR_DELAY:  200
ITR_DELAY:  30

In [32]:
# checking corr matrix
corr_dvfs_counter_list['0xd00']['200k']['300']

,instructions_diff,cycles_diff,ref_cycles_diff,llc_miss_diff
instructions_diff,1.000000,0.894238,0.893171,0.738355
cycles_diff,0.894238,1.000000,0.999599,0.803823
ref_cycles_diff,0.893171,0.999599,1.000000,0.804571
llc_miss_diff,0.738355,0.803823,0.804571,1.000000


In [33]:
# checking corr matrix
corr_dvfs_non_counter_list['0x1d00']['600k']['100']

,rx_bytes,rx_desc,tx_bytes,tx_desc
rx_bytes,1.000000,0.671676,0.006293,0.004360
rx_desc,0.671676,1.000000,0.027087,0.016394
tx_bytes,0.006293,0.027087,1.000000,0.548597
tx_desc,0.004360,0.016394,0.548597,1.000000


In [34]:
# checking joules sum values
joules_dvfs_list['0xd00']['400k']

{'300': {'joules_sum': 37753.34609300003},
 '100': {'joules_sum': 39215.126425000024},
 '400': {'joules_sum': 37394.88526599988},
 '250': {'joules_sum': 38249.420906999876},
 '200': {'joules_sum': 38266.06536700008},
 '350': {'joules_sum': 37007.07001900003}}

#### Eigenvalues of these correlation matrices:

In [37]:
eig_dvfs_list = {}
for dvfs in ['0xd00', '0x1d00']:
    eig_qps_list = {}
    for qps in ['200k', '400k', '600k']:
        eig_itr_list = {}
        for itr in corr_dvfs_counter_list[dvfs][qps].keys():
            df_corr_counter = corr_dvfs_counter_list[dvfs][qps][itr]
            df_corr_non_counter = corr_dvfs_non_counter_list[dvfs][qps][itr]
            vals_counter, vecs_counter = eig(df_corr_counter)
            vals_non_counter, vecs_non_counter = eig(df_corr_non_counter)
            i = 0
            vals_dict = {}
            for val in vals_counter:
                if val <= 0:
                    print('EIGENVAL <= 0')
                vals_dict['eig_' + str(i)] = val
                i += 1
            for val in vals_non_counter:
                if val <= 0:
                    print('EIGENVAL <= 0')
                vals_dict['eig_' + str(i)] = val
                i += 1
            eig_itr_list[itr] = vals_dict
        eig_qps_list[qps] = eig_itr_list
    eig_dvfs_list[dvfs] = eig_qps_list

In [38]:
# checking eigenvals
eig_dvfs_list['0xd00']['200k']['100']

{'eig_0': 3.540735403291909,
 'eig_1': 0.2905554291535991,
 'eig_2': 0.16866306562582573,
 'eig_3': 4.610192866839849e-05,
 'eig_4': 1.626920433151135,
 'eig_5': 1.5344901205832409,
 'eig_6': 0.37380259405780364,
 'eig_7': 0.4647868522078212}

### Latency Measurements:

In [39]:
lat_dvfs_list = {}
for dvfs in ['0xd00', '0x1d00']:
    lat_qps_list = {}
    for qps in ['200k', '400k', '600k']:
        lat_itr_list = {}
        lat_dir = qps + '_qps/linux_mcd_out_0_' + dvfs + '_135_' + qps + '/'

        for itr in corr_dvfs_counter_list[dvfs][qps].keys():
            lat_file = lat_dir + 'linux.mcd.out.0_' + str(itr) + '_' + dvfs + '_135_' + qps[:-1] + '000'
            latencies = get_latencies(lat_file)
            lat_itr_list[itr] = latencies
        lat_qps_list[qps] = lat_itr_list
    lat_dvfs_list[dvfs] = lat_qps_list

In [47]:
lat_dvfs_list['0xd00']['200k']['100']

{'avg': 117.1,
 'std': 31.0,
 'min': 49.8,
 '5th': 67.1,
 '10th': 74.9,
 '50th': 117.1,
 '90th': 157.6,
 '95th': 167.5,
 '99th': 185.0}

### Putting it all together:

In [50]:
lat_columns = []
for itr in [list(lat_dvfs_list['0xd00']['200k'].keys())[0]]:
    lat_columns = list(lat_dvfs_list['0xd00']['200k'][itr].keys())
lat_columns

['avg', 'std', 'min', '5th', '10th', '50th', '90th', '95th', '99th']

In [51]:
eig_columns = []
for itr in [list(eig_dvfs_list['0xd00']['200k'].keys())[0]]:
    eig_columns = list(eig_dvfs_list['0xd00']['200k'][itr].keys())
eig_columns

['eig_0', 'eig_1', 'eig_2', 'eig_3', 'eig_4', 'eig_5', 'eig_6', 'eig_7']

In [52]:
all_cols = lat_columns + eig_columns + ['joules_sum']
all_cols.append('desc')
print(all_cols)

['avg', 'std', 'min', '5th', '10th', '50th', '90th', '95th', '99th', 'eig_0', 'eig_1', 'eig_2', 'eig_3', 'eig_4', 'eig_5', 'eig_6', 'eig_7', 'joules_sum', 'desc']


In [53]:
eig_lat_joules_df = pd.DataFrame(columns=all_cols)
for dvfs in ['0xd00', '0x1d00']:
    for qps in ['200k', '400k', '600k']:
        for itr in corr_dvfs_counter_list[dvfs][qps].keys():
            desc = '0' + '_' + itr + '_' + dvfs + '_' + '135' + '_' + qps
            new_row = {}
            for p in lat_columns:
                new_row[p] = lat_dvfs_list[dvfs][qps][itr][p]
            for e in eig_columns:
                new_row[e] = eig_dvfs_list[dvfs][qps][itr][e]
            new_row['joules_sum'] = joules_dvfs_list[dvfs][qps][itr]['joules_sum']
            new_row['desc'] = desc
            eig_lat_joules_df = eig_lat_joules_df._append(new_row, ignore_index = True)

In [54]:
eig_lat_joules_df

,avg,std,min,5th,10th,50th,90th,95th,99th,eig_0,eig_1,eig_2,eig_3,eig_4,eig_5,eig_6,eig_7,joules_sum,desc
0,179.3,57.8,45.3,86.6,100.6,180.9,256.3,270.5,297.3,3.608221,0.258202,0.133167,4.096448e-04,1.670789,1.540596,0.354743,0.433872,34341.416562,0_200_0xd00_135_200k
1,292.5,113.5,49.8,111.7,135.7,294.9,442.9,471.3,515.1,3.593040,0.285410,0.121201,3.490791e-04,1.696883,1.574022,0.327540,0.401555,33724.280904,0_400_0xd00_135_200k
2,261.9,99.3,49.8,104.4,125.4,263.9,393.9,417.7,449.1,3.913500,0.077727,0.008757,1.586682e-05,1.690350,1.568069,0.332505,0.409076,34207.359546,0_350_0xd00_135_200k
3,117.1,31.0,49.8,67.1,74.9,117.1,157.6,167.5,185.0,3.540735,0.290555,0.168663,4.610193e-05,1.626920,1.534490,0.373803,0.464787,34939.679369,0_100_0xd00_135_200k
4,208.2,71.6,54.8,94.3,110.5,208.9,303.0,322.5,354.3,3.618876,0.256662,0.124018,4.443342e-04,1.681455,1.547267,0.345890,0.425388,34107.859213,0_250_0xd00_135_200k
5,237.6,85.6,49.8,101.1,119.6,240.1,351.6,367.0,401.8,3.573198,0.290141,0.136264,3.970409e-04,1.689835,1.562023,0.335783,0.412359,34050.928523,0_300_0xd00_135_200k
6,256.1,89.6,49.8,113.4,137.1,256.5,369.7,399.5,466.9,3.630543,0.288468,0.080902,8.643908e-05,1.736802,1.590075,0.288939,0.384184,37753.346093,0_300_0xd00_135_400k
7,139.8,46.5,45.3,81.1,90.7,136.4,186.4,207.1,285.6,3.331322,0.469135,0.199543,1.681793e-07,1.671802,1.518819,0.336722,0.472657,39215.126425,0_100_0xd00_135_400k
8,302.2,115.2,45.3,117.9,147.0,303.0,447.9,483.1,563.4,3.494643,0.401722,0.103599,3.663233e-05,1.740803,1.593680,0.294428,0.371089,37394.885266,0_400_0xd00_135_400k
9,221.8,77.4,45.3,103.5,121.5,221.6,319.3,343.0,401.3,3.186113,0.757743,0.056130,1.387622e-05,1.717363,1.592285,0.293275,0.397077,38249.420907,0_250_0xd00_135_400k


In [55]:
outdir = 'eig_lat_joules_dfs/'
eig_lat_joules_df.to_csv(outdir + '0xd00_0x1d00_200k_400k_600k')

In [58]:
df = pd.read_csv(outdir + '0xd00_0x1d00_200k_400k_600k', index_col=0)

In [59]:
df

,avg,std,min,5th,10th,50th,90th,95th,99th,eig_0,eig_1,eig_2,eig_3,eig_4,eig_5,eig_6,eig_7,joules_sum,desc
0,179.3,57.8,45.3,86.6,100.6,180.9,256.3,270.5,297.3,3.608221,0.258202,0.133167,4.096448e-04,1.670789,1.540596,0.354743,0.433872,34341.416562,0_200_0xd00_135_200k
1,292.5,113.5,49.8,111.7,135.7,294.9,442.9,471.3,515.1,3.593040,0.285410,0.121201,3.490791e-04,1.696883,1.574022,0.327540,0.401555,33724.280904,0_400_0xd00_135_200k
2,261.9,99.3,49.8,104.4,125.4,263.9,393.9,417.7,449.1,3.913500,0.077727,0.008757,1.586682e-05,1.690350,1.568069,0.332505,0.409076,34207.359546,0_350_0xd00_135_200k
3,117.1,31.0,49.8,67.1,74.9,117.1,157.6,167.5,185.0,3.540735,0.290555,0.168663,4.610193e-05,1.626920,1.534490,0.373803,0.464787,34939.679369,0_100_0xd00_135_200k
4,208.2,71.6,54.8,94.3,110.5,208.9,303.0,322.5,354.3,3.618876,0.256662,0.124018,4.443342e-04,1.681455,1.547267,0.345890,0.425388,34107.859213,0_250_0xd00_135_200k
5,237.6,85.6,49.8,101.1,119.6,240.1,351.6,367.0,401.8,3.573198,0.290141,0.136264,3.970409e-04,1.689835,1.562023,0.335783,0.412359,34050.928523,0_300_0xd00_135_200k
6,256.1,89.6,49.8,113.4,137.1,256.5,369.7,399.5,466.9,3.630543,0.288468,0.080902,8.643908e-05,1.736802,1.590075,0.288939,0.384184,37753.346093,0_300_0xd00_135_400k
7,139.8,46.5,45.3,81.1,90.7,136.4,186.4,207.1,285.6,3.331322,0.469135,0.199543,1.681793e-07,1.671802,1.518819,0.336722,0.472657,39215.126425,0_100_0xd00_135_400k
8,302.2,115.2,45.3,117.9,147.0,303.0,447.9,483.1,563.4,3.494643,0.401722,0.103599,3.663233e-05,1.740803,1.593680,0.294428,0.371089,37394.885266,0_400_0xd00_135_400k
9,221.8,77.4,45.3,103.5,121.5,221.6,319.3,343.0,401.3,3.186113,0.757743,0.056130,1.387622e-05,1.717363,1.592285,0.293275,0.397077,38249.420907,0_250_0xd00_135_400k
